### Imports modules and libs

In [1]:
import pandas as pd 
import numpy as np
from ConvertFile import Convert_File_to_pd
from DataAnalyze import DataAnalyze
from AnalyzeDuplicatedData import AnalyzeDuplicatedData
from DataTransform import DataTransform



### Transform CSVs to DFs

In [2]:
Data_frame=Convert_File_to_pd(file_name='Country',name_table='Data',folder_name='ETL-Data-FIFA')

In [3]:
Df=Data_frame.convert_to_dataframe

### exploratory analysis to investigate the possibility of data cleaning

In [4]:
#Imprimindo o DataFrame
Df

,id,name
0,1.0,Belgium
1,1729.0,England
2,4769.0,France
3,7809.0,Germany
4,10257.0,Italy
5,13274.0,Netherlands
6,15722.0,Poland
7,17642.0,Portugal
8,19694.0,Scotland
9,21518.0,Spain


In [5]:
#Classe responsável pelas análises exploratórias
Results_analysis=DataAnalyze(Df)

#### Detect NaNs

In [6]:
# No objetivo de não ferir a integridade de domínio quando estes dados forem carregados em um bd, o id não pode ter campos nulos ou NaN
Results_analysis.detect_nans(name_column='id')


'lista  indices contendo nans: [17]'

In [7]:
#Fazendo essa análise para a coluna Name, uma vez que se ela contiver NaNs, é possível deles serem tratados, pois o id possui referencia justamente a essa col
Results_analysis.detect_nans(name_column='name')

'lista  indices contendo nans: [16, 17]'

#### Detect Data Duplicated

In [8]:
#além disso, é necessário avaliar os registros duplicados quando a coluna não pode contê-los, como é o caso do ID também
Result_dup=AnalyzeDuplicatedData(dataframe=Df,name_col_data='id')


In [9]:
#Avaliando a quantidade de ids duplicados
Result_dup.print_duplicated_data(table_name='Country')

a quantidade de dados duplicados da tabela Country  na coluna id são 5


In [10]:
Result_dup.detect_indexes_data()

{11: [0, 1.0],
 12: [1, 1729.0],
 13: [2, 4769.0],
 14: [3, 7809.0],
 15: [4, 10257.0]}

In [11]:
#Agrupando os indices em que os dados estão duplicados
Result_dup.grouping_indexes()

{1.0: [0], 1729.0: [1], 4769.0: [2], 7809.0: [3], 10257.0: [4]}

In [12]:
#Agora vendo somente os valores, sem os indices
Result_dup.detect_values_data()

[1.0, 1729.0, 4769.0, 7809.0, 10257.0]

In [13]:
#Fazendo isso para coluna name 
Result_dup_name=AnalyzeDuplicatedData(dataframe=Df,name_col_data='name')

In [14]:
Result_dup_name.print_duplicated_data(table_name='Country')

a quantidade de dados duplicados da tabela Country  na coluna name são 1


In [15]:
#Como já vimos que existem dois NaNs, logo são eles os valores duplicados 
# Não necessitando ver seus indices

#### Identify Outliers

In [16]:
#Não é necessário avaliar outliers, já que não existem variáveis passíveis de análise 

### Analyze  typing

In [17]:
#analisando a tipagem esperada de forma geral
Df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      17 non-null     float64
 1   name    16 non-null     object 
dtypes: float64(1), object(1)
memory usage: 416.0+ bytes


In [18]:
#Como no método info a coluna name é object e object pode conter qualquer tipagem, então é necessário investigar se todas são Strings para serem Name
#tendo só 2 NaNs, conforme já era esperado, mas todas encontram-se como str

Results_analysis.confirm_typing(name_column='name',type_expected='str')

[nan, nan]

In [19]:
#Como os IDs númericos representam geralmente inteiros, faz-se necessário investigar se eles como float são equivalentes à inteiros
#Para isso,

Results_analysis.analyze_typing_expected_number(expected_type_number='int',name_column='id')

{0: [1.0, 'same expected value between int and float typing'],
 1: [1729.0, 'same expected value between int and float typing'],
 2: [4769.0, 'same expected value between int and float typing'],
 3: [7809.0, 'same expected value between int and float typing'],
 4: [10257.0, 'same expected value between int and float typing'],
 5: [13274.0, 'same expected value between int and float typing'],
 6: [15722.0, 'same expected value between int and float typing'],
 7: [17642.0, 'same expected value between int and float typing'],
 8: [19694.0, 'same expected value between int and float typing'],
 9: [21518.0, 'same expected value between int and float typing'],
 10: [24558.0, 'same expected value between int and float typing'],
 11: [1.0, 'same expected value between int and float typing'],
 12: [1729.0, 'same expected value between int and float typing'],
 13: [4769.0, 'same expected value between int and float typing'],
 14: [7809.0, 'same expected value between int and float typing'],
 15:

### Data transform 

In [20]:
#Instânciando o Df para a classe DataCleaning

Data_cleaning=DataTransform(dataframe_to_transform=Df)


#### Remove(or not) NaNs

In [21]:
# partindo do pressuposto de que os IDs não podem ter valores NaNs, é preciso remove-los
#Por possuirem valores distintos e não apresentam uma progressão lógica neste caso, além de serem propriamente identificadores
#Não podemos substituir por interpolação, mesmo sendo uma variável discreta

#Logo, assumiremos a remoção, haja vista que a proporção de NaNs próximos ou abaixo de 5 %  servirá como parâmetro para remoção 
Results_analysis.count_nans(name_columns=['id'])


{'id': 5.555555555555555}

In [22]:
#Como o valor ficou próximo, esses serão removidos, bem como os nomes na coluna names, os quais são correspondentes
Data_cleaning.remove_nans(name_column=['id','name'],all=False)


,index,id,name
0,0,1.0,Belgium
1,1,1729.0,England
2,2,4769.0,France
3,3,7809.0,Germany
4,4,10257.0,Italy
5,5,13274.0,Netherlands
6,6,15722.0,Poland
7,7,17642.0,Portugal
8,8,19694.0,Scotland
9,9,21518.0,Spain


#### Remove(or not) Duplicated

In [23]:
# Como os dados duplicados para os IDs não tem valores equivalentes (ex:1 Belgium , 11 Belcica), nota-se que a repetição se dá com ids cujos valores(nomes)
# Estão em Pt-Br
#Logo, removendo os ids subsequentes
Data_cleaning.remove_dup(name_column=['id'])

,index,id,name
0,0,1.0,Belgium
1,1,1729.0,England
2,2,4769.0,France
3,3,7809.0,Germany
4,4,10257.0,Italy
5,5,13274.0,Netherlands
6,6,15722.0,Poland
7,7,17642.0,Portugal
8,8,19694.0,Scotland
9,9,21518.0,Spain


#### Data Typing 

In [24]:
#Como foi visto através da geração do obj Result_analysis(juntamente com o método analyze_typing_expected_number ) de que o Id está com a tipagem float, porém é equivalente a int(.0)
#É possível transformar essa coluna em int, como é o objetivo

Data_cleaning.transf_typing(name_column='id',type_expected='int').dtypes

id       int64
name    object
dtype: object

In [25]:
# como a coluna names não identificou dados fora do tipo String, além dos Nans, então só é necessário transformar a coluna Id 

#armazenando em um novo obj os dados transformados 
Data_Transf=Data_cleaning.transf_typing(name_column='id',type_expected='int')

### Load CSV

In [26]:
Export_df_csv=Data_Transf.to_csv('Load_csvs/Country.csv')